<a href="https://colab.research.google.com/github/akshaygrao77/MABCricketProject/blob/main/Template_Cricket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import time
import math
from operator import attrgetter
np.random.seed(0)
# Students will submit their files with their team-name.py 
# Student have to use the Team as their parent class

In [ ]:
class State:
  def __init__(self,ball,total_runs,wickets_left,score_to_chase,feature_batter,feature_bowler,current_batters_list,current_bowlers_list,batting_action=None,bowling_action=None):
    self.feature_batter=feature_batter
    self.feature_bowler = feature_bowler
    self.num_ball = ball
    self.wickets_left = wickets_left
    self.bowling_action=bowling_action
    self.batting_action = batting_action
    self.current_bowlers_list = current_bowlers_list
    self.current_batters_list = current_batters_list

In [ ]:
class MCTSNode:
  def __init__(self,parent,to_play,state,isNodeStochastic = 0):
    self.visit_count = 0
    self.to_play = to_play
    # Dictionary of action,node array (bcoz sometimes, same action can lead to more than one node in case of stochastic actions)
    self.children = []
    self.state = state
    self.isNodeStochastic = 0
    self.parent = parent
    # When a state is stochastic
    self.nodeOccurenceProbability = 0
    self.valueOfNode = 0
    self.ucb_scoreOfNode = 0
  
  def expand_children(self,gameLogic):
    children_nodes = []
    possible_actions = gameLogic.get_possible_actions_in_state(self.to_play,self.state)
    children_states = gameLogic.get_children_states(self.to_play,self.state,possible_actions)
    for each_child_state in children_states:
      next_to_play = gameLogic.get_next_to_play_order(self.to_play)
      child_node = MCTSNode(parent=self,to_play=next_to_play,state=each_child_state)
      if(next_to_play == gameLogic.stochastic_role_to_play):
        child_node.isNodeStochastic = 1
      children_nodes.append(child_node)
    self.children = children_nodes
  
  def expand_stochastic_nodes(self,gameLogic):
    wicket,runs = gameLogic.environment.get_outcome(self.state.feature_batter, self.state.feature_bowler, self.state.batting_action, self.state.bowling_action)
    new_state = gameLogic.get_next_reward_state(wicket,runs,self.state)


  def select_best_child(self,round_number,gameLogic):
    for i in range(len(self.children)):
      if(self.children[i].visit_count == 0):
        return self.children[i]
    for each_child_node in self.children:
      each_child_node.compute_ucb_score(round_number)
    best_child =  max(self.children , key = attrgetter('ucb_scoreOfNode'))
    if(best_child.to_play == 'chance_reward'):
      return self.expand_stochastic_nodes(gameLogic)


  def compute_ucb_score(self,round_number):
    if(self.to_play == 'me_to_play'):
      self.ucb_scoreOfNode = (self.valueOfNode / self.visit_count) + math.sqrt(2 * round_number / self.visit_count)
    elif(self.to_play == 'opp_to_play'):
      self.ucb_scoreOfNode = (self.valueOfNode / self.visit_count) - math.sqrt(2 * round_number / self.visit_count)
  
  def simulate_random_rollout(self):
    pass

  def is_node_expanded(self):
    if(len(self.children)==0):
      return False
    return True

In [20]:
qw = np.array((1,2,3,2,5,6))
print(np.where(qw == 2)[0][0])

1


In [ ]:
class GameLogic:

  def __init__(self,environment):
    self.environment = environment
    self.to_play_order = np.array(('me_to_play','opp_to_play','chance_reward'))
    self.stochastic_role_to_play = 'opp_to_play'
    self.batting_actions = [0,1,2,3,4,6]
    self.bowling_actions = ['e','n','a']

  def get_next_reward_state(wicket,runs,previous_state):


  def get_next_to_play_order(self,current_to_play):
    next_index = np.where(self.to_play_order == current_to_play )[0][0] + 1

    if(next_index == len(self.to_play_order)):
      return self.to_play_order[0]
    return self.to_play_order[next_index]

  def get_possible_actions_in_state(self,to_play,state):
    if(to_play == 'me_to_play'):
      return self.batting_actions
    elif(to_play == 'opp_to_play'):
      return self.bowling_actions

  def get_children_states(self,to_play,state,possible_actions):
    children_states = []
    if(to_play == 'me_to_play'):
      for each_action in possible_actions:
        child_state = State(feature_batter=state.feature_batter,feature_bowler=state.feature_bowler,num_ball=state.num_ball,wicket=state.wicket,batting_action=each_action)
        children_states.append(child_state)
    elif(to_play == 'opp_to_play'):
      for each_action in possible_actions:
        child_state = State(feature_batter=state.feature_batter,feature_bowler=state.feature_bowler,num_ball=state.num_ball,wicket=state.wicket,batting_action=state.batting_action,bowling_action=each_action)
        children_states.append(child_state)
    return children_states
  



In [ ]:
class MCTSAlgo:
  def __init_(self,environment):
    self.environment = environment;

  def run(self,state,environment):
    gameLogic = GameLogic(environment)
    root = MCTSNode(None,'me_to_play',state)

    root.expandChildren(gameLogic)



In [ ]:
class Team:
  def __init__(self,self_features,opponent_features,environment):
    self.self_features = self_features
    self.opponent_features = opponent_features
    self.explore_wicket = 0
    self.explore_runs = 0
    self.environment = environment
    self.current_batters_list =np.array([1,1,1,1,1])  # 0 indicates player who had got out, -1 indicates currently playing bowler in current state, 1 indicates bowler yet to bat
    self.current_bowlers_list =np.array([2,2,2,2,2]) # -number indicates currently playing bowler, +ve number indicates number of overs left

    self.batting_order = [0,1,2,3,4]

  # Returns index of player who is not-out as per batting order
  def get_next_batter(self):
    for i in range(len(self.batting_order)):
      # If batter is currently batting or already out, skip him
      if(self.current_batters_list[i] > 0):
        return i

  # For phase 2
  # def get_next_bowler(self):
  #   return np.random.randint(0,5)

  def get_batting_action(self,ball,total_runs,wickets_left,score_to_chase,feature_batter,feature_bowler):
    batter = -1
    for index_of_each_full_feature in range(len(self.self_features)):
      if(self.self_features[index_of_each_full_feature,0:2] == feature_batter):
        batter = index_of_each_full_feature
    previous_batter_index = np.where(self.current_batters_list == -1)[0][0]
    self.current_batters_list[previous_batter_index] = 0
    self.current_batters_list[batter] = -1

    for index_of_each_full_feature in range(len(self.opponent_features)):
      if(self.opponent_features[index_of_each_full_feature,2:4] == feature_bowler):
        bowler = index_of_each_full_feature
    previous_bowler_index = np.where(self.current_bowlers_list < 0)[0][0]
    self.current_bowlers_list[previous_bowler_index] = -(self.current_bowlers_list[previous_bowler_index] + 1)
    self.current_bowlers_list[bowler] = -(self.current_bowlers_list[bowler])

    batting_action = np.random.randint(0,6)  # this is the place to code UCTS
    return batting_action

  def get_bowling_action(self,ball,total_runs,wickets_left,score_to_chase,feature_batter,feature_bowler):
    bowling_action = np.random.randint(0,3) # this is the place to code UCTS
    return bowling_action

  def explore(self):
    pass

  


class Australia(Team):
  pass

class India(Team):
  pass


In [ ]:
class Environment: 
  def get_outcome(self,feature_batter, feature_bowler, batting_action, bowling_action):
    wicket = np.random.randint(0,1)
    runs = np.random.randint(0,7)
    return wicket, runs

class Match:

  def __init__(self,num_balls, explore_num_balls,action_timeout, explore_timeout, TeamOne, TeamTwo):
    self.environment = Environment()
    self.num_balls = num_balls
    self.explore_num_balls = explore_num_balls
    self.action_timeout = action_timeout          # time limit for any given act
    self.explore_timeout = explore_timeout        # time limit for exploration phase
    self.team_one_features = np.random.uniform(1,5,size=(5,4))   #feature[0] : batting average, feature[1]: strike-rate, feature[2]: bowling average, feature[3]: economy
    self.team_two_features = np.random.uniform(1,5,size=(5,4))
    self.team_one = TeamOne(self.team_one_features,self.team_two_features,self.environment)
    self.team_two = TeamTwo(self.team_two_features,self.team_one_features,self.environment)
    self.current_batters_list = np.array([1,1,1,1,1])  # a coordinate is set to 0 when that corresponding batter gets out, if the third player is out then you have [1,1,0,1,1]
    self.current_bowlers_list = np.array([2,2,2,2,2])  # if the 4th bowler bowls the first over, after first over we have [2,2,2,1,2].
    self.num_miss_team_batting = 0
    self.num_miss_team_bowling = 0


  def explore_phase_team(self,team_id):    #timed phase
    if (team_id == 1):
      team = self.team_one
    else:
      team = self.team_two
    start_time      = time.time()
    team.explore()    
    end_time = time.time()    
    if(end_time - start_time > self.explore_timeout):  
      print("Timing Violation During Exploration Phase")
      

  def explore_phase(self):     #allow each of the teams to explore
    self.explore_phase_team(1)
    self.explore_phase_team(2)


  def get_valid_bowler(self,next_bowler):
    if (self.current_bowlers_list[next_bowler]==0):
      print("Bowler Invalid, Choosing Random Bowler")
      bowlers_with_overs_left = np.where(self.current_bowlers_list>0)[0]
      # next_bowler = np.random.choice(bowlers_with_overs_left) # For phase 2
      next_bowler = bowlers_with_overs_left[0]  # Sequentially send bowlers
    return next_bowler


  def get_valid_batter(self,next_batter):
    if (self.current_batters_list[next_batter]==0):
      print("Batter Invalid, Choosing Random Batter")
      batters_not_out = np.where(self.current_batters_list>0)[0]
      next_batter = np.random.choice(batters_not_out)
    return next_batter

  def next_batter(self,team_batting):
    next_batter = team_batting.get_next_batter()
    next_batter = self.get_valid_batter(next_batter)
    feature_batter = team_batting.self_features[next_batter,0:2]
    return next_batter,feature_batter


  def next_bowler(self,team_bowling):
    # next_bowler = team_bowling.get_next_bowler() # For phase 2
    next_bowler = self.get_valid_bowler(0)
    feature_bowler = team_bowling.self_features[next_bowler,2:4]
    return next_bowler, feature_bowler

  def get_team_batting_action(self,team_batting,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler):
    start_time      = time. time()
    batting_action  = team_batting.get_batting_action(ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler)
    end_time        = time. time()
    if(end_time - start_time > self.action_timeout):
      batting_action = 0 #this is the default option, we have to fix the penalisation strategy
      self.num_miss_team_batting = self.num_miss_team_batting + 1
    return batting_action


  def get_team_bowling_action(self,team_bowling,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler):
    start_time      = time. time()
    bowling_action  = team_bowling.get_bowling_action(ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler)
    end_time        = time. time()
    if(end_time - start_time > self.action_timeout):
      bowling_action = 0 #this is the default option, we have to fix the penalisation strategy
      self.num_miss_team_bowling = self.num_miss_team_bowling + 1
    return bowling_action



  def innings(self,innigins_id,runs_to_chase):
    total_runs = 0
    wickets_left = 5
    self.current_batters_list =np.array([1,1,1,1,1])
    self.current_bowlers_list =np.array([2,2,2,2,2])
    self.num_miss_team_batting = 0
    self.num_miss_team_bowling = 0
    if (innigins_id == 1 ):
      team_batting = self.team_one
      team_bowling = self.team_two
    else:
      team_batting = self.team_two
      team_bowling = self.team_one
    # Initialising the first batter and first bowler
    batter, feature_batter = self.next_batter(team_batting)
    bowler, feature_bowler = self.next_bowler(team_bowling)
    
    for ball in range(self.num_balls):
      if np.sum(self.current_batters_list) > 0 :
        batting_action = self.get_team_batting_action(team_batting,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler)
        bowling_action = self.get_team_bowling_action(team_bowling,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler)
        wicket, runs   = self.environment.get_outcome_random(feature_batter, feature_bowler, batting_action, bowling_action)
        total_runs     = total_runs + runs
        if (wicket > 0):
          self.current_batters_list[batter] = 0
          wickets_left = wickets_left - 1
          if(np.sum(self.current_batters_list) > 0 ):
            batter,feature_batter = self.next_batter(team_batting)
        if ((ball+1)%6 ==0 ):
          self.current_bowlers_list[bowler] = self.current_bowlers_list[bowler]-1
          if(np.sum(self.current_bowlers_list) > 0 ) :
            bowler, feature_bowler = self.next_bowler(team_bowling)

    return total_runs, self.current_batters_list, self.current_bowlers_list, self.num_miss_team_batting, self.num_miss_team_bowling


In [ ]:

# setting time outs
explore_timeout = 10
action_timeout = 10

explore_num_balls = 60 # this number will be much lesser when we actually test the code.
num_balls = 60         # fixed
match = Match(num_balls, explore_num_balls,action_timeout, explore_timeout,Australia,India)
match.explore_phase()
first_innings_score, batters_list, bowlers_list, num_miss_team_batting, num_miss_team_bowling = match.innings(1,float('inf'))
second_innings_score, batters_list, bowlers_list, num_miss_team_batting, num_miss_team_bowling = match.innings(2,first_innings_score)